# Applied Data Science Capstone - Week 3 Assignment - Question 2

##### Scrape postal codes information for Toronto from Wikipedia:

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

test = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(test, 'lxml')
table = soup.find('table')

m = table.tbody.tr.text
y = m.split('\n')
z = table.tbody.text
p = y = z.split('\n')
p = [i for i in p if i] 
p2 = [p[x:x+3] for x in range(0, len(p), 3)]
header = p2[0]
del p2[0] 
p3 = pd.DataFrame(p2, columns = header)

In [2]:
print(p3.shape)

(288, 3)


In [4]:
p3.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### Remove boroughs with values of "Not assigned":

In [5]:
p4 = p3[p3.Borough != 'Not assigned']

##### Combine Neighbourhood value for Postal codes with more than one Neigbourhood value separated with a comma:

In [6]:
p5 = p4.groupby("Postcode")["Borough"].count().reset_index()
p5 = list(p5[p5.Borough > 1].Postcode)
p6 = p4[p4.Postcode.isin(p5)][['Postcode','Neighbourhood']].sort_values(["Postcode"]).reset_index(drop = True)
Neighbourhood = []
pc = p6.iloc[0]['Postcode']
nn = p6.iloc[0]['Neighbourhood']
p6.drop([0], axis = 0, inplace = True)
for i, j in zip(p6.Postcode, p6.Neighbourhood):
    if pc == i:
        nn = nn + ", " + j
    else:
        pc = i
        Neighbourhood.append(nn)
        nn = j
Neighbourhood.append(nn)
        
Postcode = list(p6.Postcode.unique()) 
        
print(len(Neighbourhood))
print(len(Postcode))

57
57


In [7]:
new_df = pd.DataFrame(columns = ['Postcode','Neighbourhood'])
new_df['Postcode'] = Postcode
new_df['Neighbourhood'] = Neighbourhood

Borough_lookup = p4[p4.Postcode.isin(Postcode)][['Postcode','Borough']].drop_duplicates().reset_index(drop = True)
clean_df = pd.merge(new_df, Borough_lookup, on = 'Postcode')[['Postcode','Borough','Neighbourhood']]

p4.drop(p4.loc[p4.Postcode.isin(Postcode)].index, axis = 0, inplace = True)
p4.reset_index(drop = True, inplace = True)

final_df = pd.concat([p4, clean_df], axis = 0)
final_df.reset_index(drop = True, inplace = True)
final_df = final_df.sort_values('Postcode', ascending = True).reset_index(drop = True)

final_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Repace value of Neighbourhood with it's equivalent Borough value if it's current Neighbourhood value is "Not assigned":

In [8]:
final_df.loc[final_df.Neighbourhood == "Not assigned", 'Neighbourhood'] = final_df.loc[final_df.Neighbourhood == "Not assigned"].Borough

final_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
print(final_df.shape)

(103, 3)


##### Load Geospatial Coordinates Data on a dataframe:

In [10]:
df_geocoord = pd.read_csv("geospatial_coordinates.txt", "\t")
df_geocoord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Combine the dataframes to create a single final dataframe for analysis:

In [12]:
df_capstone_f = pd.merge(final_df, df_geocoord, left_on = "Postcode", right_on = "Postal Code")[["Postcode","Borough","Neighbourhood","Latitude","Longitude"]]
df_capstone_f.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
